# Exercise 1: Function calling with the OpenAI API

Large Language Models (LLMs) are incredibly powerful. But, as we have seen, they can struggle with simple mathematics problems and are limited to the information captured in the training data.

To this end, the Chat Completion API now allows us to describe functions and tools to the model. The models themselves won't call the functions, but they can determine if they are needed and return the arguments to use.

In this notebook you will learn how describing functions to a model can transform it into agent, which is able to reason with itself to answer a query.

In [ ]:
# autoreload imports
%load_ext autoreload

%autoreload 2

In [ ]:
from llm_in_production.openai_utils import get_openai_client
from llm_in_production.agent_utils import (
    function_to_json,
    function_calling_agent,
)
import dotenv
import numpy as np
import os
import json
import requests
import pandas as pd
import yfinance as yf
import tiktoken

# This reads the .env file in your project and transforms its content into env variables.
# This way you don't have to hard code your secrets.
dotenv.load_dotenv()
# Here we create the client.
client = get_openai_client()

## LLM limiations

Let's start by demonstrating one of the limitations of LLMs.

The prompt below is asking the model to answer a simple highschool mathematics problem.

In [ ]:
math_problem = "what is the circumference of a circle with radius 5.31cm"

In [ ]:
system_prompt = "You are a calculator bot that is used to answer mathematics problems"

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": math_problem},
]


In [ ]:

response = client.chat.completions.create(
    model=os.environ["GPT_4_MODEL_NAME"],
    messages=messages,
    seed=0,
)

message = response.choices[0].message.content

print(message)

However, the answer it produces is incorrect.

$2*\pi*5.31 = 33.3637139811...$

For this problem, a simple function would be more useful!

In [ ]:
def circumference_calculator(radius: float, something: float = 4.4) -> float:
    """Calculates the circumference of a circle given the radius

    :param radius: The radius of the circle
    :return: The circumference of the circle
    """
    return 2 * np.pi * radius


circumference_calculator(5.31)

But what if GPT knew about this function?

## Function Calling

The Chat Completion API allows us to describe functions and tools to the model. The model can then decide when it is appropriate to use one of these functions and if so, return the arguments for the function.

To describe a function to the model it must be in a particular JSON format. We have ceated a helpful function to transform python functions into this format.

In [ ]:
circumference_calculator_json = function_to_json(circumference_calculator)
circumference_calculator_json

With this function described to the model, the response to the previous prompt is that the function should be called with the argument `radius=5.31`.

In [ ]:
tools = [{"type": "function", "function": circumference_calculator_json}]

response = client.chat.completions.create(
    model=os.environ["GPT_4_MODEL_NAME"],
    messages=messages,
    seed=0,
    tools=tools,
    tool_choice = "auto"  # (default setting) the model will pick between generating a message or calling a function automatically 
)

response_message = response.choices[0].message
message = response_message.content
tool_calls = response_message.tool_calls

print(f"Message: {message}")
print(f"Tool calls: {tool_calls}")

Let's do what the model suggests and call the function with the given argument.

In [ ]:
function_name = tool_calls[0].function.name
function_args = json.loads(tool_calls[0].function.arguments)
function_response = json.dumps(eval(function_name)(**function_args))

print(f"Fucntion response {function_response}")


The number is correct. Let's pass information back to the model.

First we need to update the message history.

In [ ]:
messages.append(response_message)

messages.append(
    {
        "role": "tool",
        "name": function_name,
        "content": function_response,
        "tool_call_id": tool_calls[0].id,
    }
)  # extend conversation with function response

if len(messages) > 4:
    raise Exception(
        "Too many messages have been added! Restart and rerun the notebook."
    )

messages

Now we can call the model again to answer the original query.

In [ ]:
response = client.chat.completions.create(
    model=os.environ["GPT_4_MODEL_NAME"],  # need a model version 0613 or higher
    messages=messages,
    seed=0,
    tools=tools,
)  # get a new response using the result of the function call


message = response.choices[0].message.content
tool_calls = response.choices[0].message.tool_calls

print(f"New tool calls: {tool_calls}")
print(message)

Thanks to calling the function, we were able to produce the correct result!

Notice how the model will only reach for a function if it is needed.

In [ ]:
history_problem = "How many wives did king Henry VIII of England have?"

response = client.chat.completions.create(
    model=os.environ["GPT_4_MODEL_NAME"],  # need a model past 0613
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": history_problem},
    ],
    seed=0,
    tools=tools,
)

response_message = response.choices[0].message
message = response_message.content
tool_calls = response_message.tool_calls

print(f"Message: {message}")
print(f"Tool calls: {tool_calls}")

## External APIs

One way of gathering live information is through the use of an external API.

They can enable us to retrieve a wide variety of information (although some will come at a cost for commercial usage). For example:
- The weather conditions for a particular location.
- The topic headline from a particular news source.
- The recent prices of a stock .


### Weather

To get information about local weather we can use the API from https://www.weatherapi.com, which is availble on a [free plan](https://www.weatherapi.com/pricing.aspx). An API key has been provided for you in the Codespace.

We can use the function below to obtain weather information for a particular location. You can take a look at the [WeatherAPI docs](https://www.weatherapi.com/docs/) for more examples of how to use the API.

In [ ]:
def get_current_weather(location: str) -> dict:
    """Get the current weather conditions in a given location

    :param location: The city (and state), e.g. "San Francisco, CA"
    :return: The weather conditions
    """
    return requests.get(
        f"https://api.weatherapi.com/v1/current.json?key={os.environ['WEATHER_API_KEY']}&q={location}"
    ).json()


get_current_weather("London")

There is a lot of information in the JSON that the function returns, more than is needed for the average person. However, a LLM would be able to summarize this information into a readable paragrpah.

In the cell below we have written a system prompt and user prompt, which we pass to the function `function_calling_agent`, along with the `get_current_weather` function. 

The `function_calling_agent` is a helpful function we wrote to automate the reasoning process used above:
- First, any functions we want to give the model access to are transformed into the JSON format it expects.
- We then extract a response from a model, based on the input prompts. 
- The process may finish there, but if the model deems if necessary, it may ask for a function to be called. In this case:
    - A function is called using the parameters returned by the model,
    - The function response is then given to the model.
    - Finally, the output to this new infromation is returned.

We have added some print statements to help you understand what is going on "under the hood".


In [ ]:
system_prompt = """
You are a weather bot that is able to give back a summary of the weather conditions in the given location. 

You do not return many measurements about the conditions.

Instead, you give an overall idea as to what conditions are like in one or two sentences."
"""

user_prompt = "What is the weather in London?"

In [ ]:
output = function_calling_agent(client, system_prompt, user_prompt, get_current_weather)
print(output)

### Exercise 1a: News

Your task is to build an agent that can summarize the top headlines of the day from a given news source.

To give the model access to the latest news stories, you can use the https://newsapi.org/ API, which is available on a [free plan](https://newsapi.org/pricing). An API key is already provided for you in the Codespace.


#### Part i: The day's top headlines

Write a Python function that returns JSON information about the top headlines of the day (from a given source) in the Python dictionary format. You can use the examples given in the [NewsAPI docs](https://newsapi.org/docs/endpoints/top-headlines) to help you answer this question.

In [ ]:
def get_top_headlines(source: str) -> dict:
    # YOUR CODE HERE START
    # YOUR CODE HERE END

get_top_headlines("bbc-news")

#### Part ii: Prompt engineering for a news summary

Write a system prompt and user prompt that will lead the agent to summarize the day's top headlines. 

Rather than simply returning a list of the top stories, the agent should be able to condense the information into a few sentences. The agent should also be capable of only writing about the news category the user is interested in, e.g. politics or sport.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

In [ ]:
output = function_calling_agent(client, system_prompt, user_prompt, get_top_headlines)
print(output)

### Exercise 1b: Stocks

In this exercise you will build a stock analysis agent to help a user to make investment decisions.

The agent will be able examine the recent history of a stock, along with recent press articles about it, to determine whether or not the stock is a good investment.

<mark>Warning! This agent is only intended to be used for educational purpose and we do not expect its suggestions to be of much worth.</mark> 

<mark> Xebia will not be held liable for any financial losses incurred from its usage!</mark> 

To get the recent history of a stock we will use [yfinance](https://github.com/ranaroussi/yfinance). yfinance offers a threaded and Pythonic way to download market data from [Yahoo!Ⓡ finance](https://finance.yahoo.com). Note that it is **not** affiliated, endorsed, or vetted by Yahoo, Inc. It's an open-source tool that uses Yahoo's publicly available APIs, and is intended for research and educational purposes.

Given a stock ticker symbol, we can retrive recent information about that stock.

In [ ]:
stock = yf.Ticker("IBM")
df = stock.history(period=f"30d")
df = df[["Close", "Volume"]]
df.index = [str(x).split()[0] for x in list(df.index)]
df.index.rename("Date", inplace=True)
df

#### Part i: Recent stock history

Write a Python function that returns JSON information about the recent history of a particular stock in the Python dictionary format.

In [ ]:

def get_stock_prices(company_stock_ticker_symbol: str, days: int = 30) -> dict:
    # YOUR CODE HERE START
    # YOUR CODE HERE END
    return df.to_json()

In [ ]:
get_stock_prices("IBM")

We can use NewsAPI again to retrieve the recent headlines about a particular stock.

In [ ]:
today = pd.Timestamp.today()
start = pd.Timestamp.today()
end = today - pd.Timedelta(days=30)

In [ ]:
topic = "IBM stock news"

request = requests.get(
    f"https://newsapi.org/v2/everything?q={topic}&from={start}&to={end}&sortBy=popularity&apiKey={os.environ['NEWS_API_KEY']}"
).json()
articles = pd.DataFrame(request["articles"])
titles = articles["title"]
titles


#### Part ii: Any recent news?

Write a Python function that returns JSON information about the recent headlines of a particulaer stock in the Python dictionary format.

In [ ]:
def get_news_stories(topic: str) -> dict:
    # YOUR CODE HERE START
    # YOUR CODE HERE END
    return titles.to_json()

In [ ]:
get_news_stories("Apple")

#### Part iii: Prompt engineering for a stock

Write a system prompt and user prompt that will lead the agent to determine whether or not a stock is a good investment. 

As you may have already experienced with ChatGPT, the model will want to caveat its opinions.
For the purpose of this assignment, see if you can get it to provide an opinion without any reservations.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

In [ ]:
output = function_calling_agent(client, system_prompt, user_prompt, get_news_stories, get_stock_prices)
print(output)

#### Part iv: Prompt engineering for multiple stocks.

Update your system/user prompt so that it will lead the agent to make an investment decision for more than one stock.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

---